In [18]:
from selenium import webdriver
from bs4 import BeautifulSoup
import pandas as pd

In [28]:
import mysql.connector  as sc
myconn = sc.connect(
    database = 'adbms',
    host = 'localhost',
    user = 'root',
    password ='Kushashu123'
)
print(myconn)
cursor = myconn.cursor(buffered = True)

In [2]:
!pip install selenium

     ---------------------------------------- 6.5/6.5 MB 982.8 kB/s eta 0:00:00
     ------------------------------------ 384.9/384.9 kB 887.9 kB/s eta 0:00:00
     -------------------------------------- 58.3/58.3 kB 607.8 kB/s eta 0:00:00


In [9]:
!pip install beautifulsoup4

In [21]:
from bs4 import BeautifulSoup
import requests
import re
import pandas as pd


# Downloading imdb top 250 movie's data
url = 'http://www.imdb.com/chart/top'
response = requests.get(url)
soup = BeautifulSoup(response.text, "html.parser")
movies = soup.select('td.titleColumn')
crew = [a.attrs.get('title') for a in soup.select('td.titleColumn a')]
ratings = [b.attrs.get('data-value')
		for b in soup.select('td.posterColumn span[name=ir]')]

# create a empty list for storing
# movie information
list = []

# Iterating over movies to extract
# each movie's details
for index in range(0, len(movies)):
	
	# Separating movie into: 'place',
	# 'title', 'year'
	movie_string = movies[index].get_text()
	movie = (' '.join(movie_string.split()).replace('.', ''))
	movie_title = movie[len(str(index))+1:-7]
	year = re.search('\((.*?)\)', movie_string).group(1)
	place = movie[:len(str(index))-(len(movie))]
	data = {"place": place,
			"movie_title": movie_title,
			"rating": ratings[index],
			"year": year,
			"star_cast": crew[index],
			}
	list.append(data)

# printing movie details with its rating.
for movie in list:
	print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +
		') -', 'Starring:', movie['star_cast'], movie['rating'])


##.......##
df = pd.DataFrame(list)
df.to_csv('imdb_top_250_movies.csv',index=False)


1 - The Shawshank Redemption (1994) - Starring: Frank Darabont (dir.), Tim Robbins, Morgan Freeman 9.235857621446582
2 - The Godfather (1972) - Starring: Francis Ford Coppola (dir.), Marlon Brando, Al Pacino 9.155772967990414
3 - The Dark Knight (2008) - Starring: Christopher Nolan (dir.), Christian Bale, Heath Ledger 8.991957602034253
4 - The Godfather Part II (1974) - Starring: Francis Ford Coppola (dir.), Al Pacino, Robert De Niro 8.983807982931284
5 - 12 Angry Men (1957) - Starring: Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb 8.953460266380482
6 - Schindler's List (1993) - Starring: Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes 8.940410970601459
7 - The Lord of the Rings: The Return of the King (2003) - Starring: Peter Jackson (dir.), Elijah Wood, Viggo Mortensen 8.932883500017175
8 - Pulp Fiction (1994) - Starring: Quentin Tarantino (dir.), John Travolta, Uma Thurman 8.846012842871843
9 - The Lord of the Rings: The Fellowship of the Ring (2001) - Starring: Peter Jackson (d

In [22]:
data = {"place": place,
            "movie_title": movie_title,
            "rating": ratings[index],
            "year": year,
            "star_cast": crew[index],
            }
	print(movie['place'], '-', movie['movie_title'], '('+movie['year'] +
		') -', 'Starring:', movie['star_cast'], movie['rating'])

In [36]:
query = """drop table imdb_250"""
cursor.execute(query)

In [37]:
query = """create table imdb_250(
id Integer,
movie_title varchar(100),
rating float,
year Integer,
star_cast varchar(100))"""
cursor.execute(query)

In [38]:
query = """insert into imdb_250 values(%s,%s,%s,%s,%s)"""
for movie in list:
    cursor.execute(query,(movie['place'],movie['movie_title'],movie['rating'],movie['year'],movie['star_cast']))

In [39]:
query = """select * from imdb_250"""
cursor.execute(query)
for x in cursor:
    print(x)

(1, 'The Shawshank Redemption', 9.23586, 1994, 'Frank Darabont (dir.), Tim Robbins, Morgan Freeman')
(2, 'The Godfather', 9.15577, 1972, 'Francis Ford Coppola (dir.), Marlon Brando, Al Pacino')
(3, 'The Dark Knight', 8.99196, 2008, 'Christopher Nolan (dir.), Christian Bale, Heath Ledger')
(4, 'The Godfather Part II', 8.98381, 1974, 'Francis Ford Coppola (dir.), Al Pacino, Robert De Niro')
(5, '12 Angry Men', 8.95346, 1957, 'Sidney Lumet (dir.), Henry Fonda, Lee J. Cobb')
(6, "Schindler's List", 8.94041, 1993, 'Steven Spielberg (dir.), Liam Neeson, Ralph Fiennes')
(7, 'The Lord of the Rings: The Return of the King', 8.93288, 2003, 'Peter Jackson (dir.), Elijah Wood, Viggo Mortensen')
(8, 'Pulp Fiction', 8.84601, 1994, 'Quentin Tarantino (dir.), John Travolta, Uma Thurman')
(9, 'The Lord of the Rings: The Fellowship of the Ring', 8.81133, 2001, 'Peter Jackson (dir.), Elijah Wood, Ian McKellen')
(1, ' Il buono, il brutto, il cattivo', 8.78864, 1966, 'Sergio Leone (dir.), Clint Eastwood, E